In [41]:
# Import the required libraries
import pandas as pd
import numpy as np

In [42]:
# Load the data 
df = pd.read_csv('SaYoPillow.csv')
df.head()

,sr,rr,t,lm,bo,rem,sr.1,hr,sl
0,93.80,25.680,91.840,16.600,89.840,99.60,1.840,74.20,3
1,91.64,25.104,91.552,15.880,89.552,98.88,1.552,72.76,3
2,60.00,20.000,96.000,10.000,95.000,85.00,7.000,60.00,1
3,85.76,23.536,90.768,13.920,88.768,96.92,0.768,68.84,3
4,48.12,17.248,97.872,6.496,96.248,72.48,8.248,53.12,0


In [43]:
# Define the calculate entropy function
def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
                        for i in range(len(classes))])
    return entropy_value


In [44]:
# Define the calculate information gain function
def calculate_information_gain(dataset,feature,label): 
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])   
    values,feat_counts= np.unique(dataset[feature],return_counts=True)
    
    # Calculate the weighted feature entropy                                # Call the calculate_entropy function
    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]
                              ==values[i]).dropna()[label]) for i in range(len(values))])    
    feature_info_gain = dataset_entropy - weighted_feature_entropy
    return feature_info_gain

In [45]:
# Set the features and label
features = df.columns[:-1]
label = 'sl'
parent=None
features

Index(['sr', 'rr', 't', 'lm', 'bo', 'rem', 'sr.1', 'hr'], dtype='object')

In [46]:
def create_decision_tree(dataset, df, features, label, parent):
    
    datum = np.unique(df[label],return_counts=True)
    unique_data = np.unique(dataset[label])
    
    if len(unique_data) <=1:
        return unique_data[0]
    elif len(dataset) == 0:
        return unique_data[np.argmax(datum[1])]
    elif len(features) == 0:
        return parent
    else:
        parent = unique_data[np.argmax(datum[1])]
        
        
        item_values = [calculate_information_gain(dataset, feature, label) for feature in features]
        
        optimum_feature_index = np.argmax(item_values)
        optimum_feature = features[optimum_feature_index]
        decision_tree = {optimum_feature:{}}
        features = [i for i in features if i != optimum_feature]
        
        for value in np.unique(dataset[optimum_feature]):
            min_data = dataset.where(dataset[optimum_feature] == value).dropna()
            
            min_tree = create_decision_tree(min_data, df, features, label, parent)
            
            decision_tree[optimum_feature][value] = min_tree
            
        return(decision_tree)

In [47]:
def predict_stress(test_data, decision_tree):
    
    for nodes in decision_tree.keys():
        value = test_data[nodes]
        decision_tree = decision_tree[nodes][value]
        
        prediction = 0
        if type(decision_tree) is dict:
            
            prediction = predict_stress(test_data, decision_tree)
        else:
            prediction = decision_tree
            break;
    return prediction

In [48]:
# Train the decision tree model
decision_tree = create_decision_tree(df,df,features,label,parent)

In [51]:
# Predict using the trained model
sample_data = {'sr':60.00,'rr':20.00,'t':96.00,'lm':10.00,'bo':95.00,'rem':85.00,'sr.1':7.000,'hr':60.00}
test_data = pd.Series(sample_data)
prediction = predict_stress(test_data,decision_tree)
prediction

1.0